<a href="https://colab.research.google.com/github/tornikeo/lut-gpgpu-course/blob/main/gpgpu_e2_tornikeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.1 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661205 sha256=dfcae5a99670d9056effc630ab412c0cc985a926ecd93caf8e44ada1acac9889
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


In [ ]:
from math import ceil as roof
import pycuda
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import numpy as np

## Task 1:

a) Implement a kernel which takes two vectors A and B and adds them together to form a vector C.

In [ ]:
size = int(2 ** 20)
a = np.random.uniform(size=size)
b = np.random.uniform(size=size)
r = np.empty_like(a)

In [ ]:
a_cu = cuda.mem_alloc(a.nbytes)
b_cu = cuda.mem_alloc(b.nbytes)
r_cu = cuda.mem_alloc(r.nbytes)

In [ ]:
kernel = SourceModule("""
__global__ void kernel(double* a, double* b, double* r, size_t n) {
  auto th = threadIdx.x + blockIdx.x * blockDim.x;
  if (th < n) {
    r[th] = a[th] + b[th];
  }
}
""").get_function('kernel')

In [ ]:
cuda.memcpy_htod(a_cu, a)
cuda.memcpy_htod(b_cu, b)
cuda.memcpy_htod(r_cu, r)
n = np.uint64(len(a))

THREADS_PER_BLOCK = (1024,1,1)
BLOCKS_PER_GRID = (roof(n / THREADS_PER_BLOCK[0]), 1, 1)

kernel(a_cu, b_cu, r_cu, n,
       block=THREADS_PER_BLOCK,
       grid=BLOCKS_PER_GRID,)

cuda.memcpy_dtoh(r, r_cu)

assert np.allclose(a + b, r)

## Task 2:

Implement a kernel which multiplies two matrices together.

The kernel function call should be in the following form:
```
__global__ void matrix_multiplication(const double* A, const double* B, double* C, int N, int M, int K)
```
  Where:
    N - number of rows for A
    M - number of columns for A
    K - number of columns for B

In [ ]:
from pycuda import gpuarray

N = np.int32(512)
M = np.int32(1024)
K = np.int32(1024)

a = np.random.normal(size=(N,M))
b = np.random.normal(size=(M,K))
c = np.empty(shape=(N,K),dtype=a.dtype)

a_cu = gpuarray.to_gpu(a)
b_cu = gpuarray.to_gpu(b)
c_cu = gpuarray.empty((N,K), np.float64)

In [ ]:
kernel = SourceModule("""
__global__ void kernel(double* a, double* b, double* c,
    int N, int M, int K) {
    auto th_x = threadIdx.x + blockIdx.x * blockDim.x;
    auto th_y = threadIdx.y + blockIdx.y * blockDim.y;
    if (th_x < N && th_y < K) {
      double s = 0;
      for(auto i = 0; i < M; i++) {
        s += (a[th_x * M + i] * b[i * K + th_y]);
      }
      c[th_x * K + th_y] = s;
    }
}
""").get_function('kernel')

THREADS_PER_BLOCK = 1,1,1
BLOCKS_PER_GRID = roof(N / THREADS_PER_BLOCK[0]), \
                  roof(M / THREADS_PER_BLOCK[0]), \
                  1

kernel(
    a_cu, b_cu, c_cu, N, M, K,
    block=THREADS_PER_BLOCK,
    grid=BLOCKS_PER_GRID,
)

assert np.allclose(a @ b, c_cu.get())

/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()
/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()


## Task 3:

Extend the kernel from task 2 to use shared memory.

In [ ]:
c_cu = gpuarray.empty((N,K), np.float64)

# I'm stuck here...

kernel = SourceModule("""
__global__ void kernel(double* a, double* b, double* c,
    int N, int M, int K) {

    const auto B = blockDim.x;

    auto tx = threadIdx.x,
         ty = threadIdx.y;
    auto bx = tx + blockIdx.x * blockDim.x,
         by = ty + blockIdx.y * blockDim.y;
    auto gx = gridDim.x,
         gy = gridDim.y;

    __shared__ double blockA[B][B], blockB[B][B];

    auto a_block_start = <stuck here>
    auto b_block_start = <arrrgh>

    double s = 0;
    if (bx < N && by < K) {
      for (int k = 0; k < gridDim.y; k++)
      {
        blockA[tx][ty] = a[bx * N + k*B + ty];
        blockB[ty][tx] = b[by + (k*B + tx) * N];
        __syncthreads();
        for(int kk = 0; kk < B; kk++) {
          s += a[kk][tx] * b[ty][kk];
        }
        __syncthreads();

      }
      c[bx][by] = s;
    }
}
""").get_function('kernel')

THREADS_PER_BLOCK = 1,1,1
BLOCKS_PER_GRID = roof(N / THREADS_PER_BLOCK[0]), \
                  roof(M / THREADS_PER_BLOCK[0]), \
                  1

kernel(
    a_cu, b_cu, c_cu, N, M, K,
    block=THREADS_PER_BLOCK,
    grid=BLOCKS_PER_GRID,
)

assert np.allclose(a @ b, c_cu.get())

/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()
<ipython-input-20-b3f01b29c519>:4: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(9): warning #177-D: variable "bx" was declared but never referenced
      auto bx = tx + blockIdx.x * blockDim.x,
           ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

kernel.cu(10): warning #177-D: variable "by" was declared but never referenced
           by = ty + blockIdx.y * blockDim.y;
           ^

kernel.cu(11): warning #177-D: variable "gx" was declared but never referenced
      auto gx = gridDim.x,
           ^

kernel.cu(12): warning #177-D: variable "gy" was declared but never referenced
           gy = gridDim.y;
           ^

kernel.cu(14): warning #177-D: variable "as" was declared but never referenced
      __attribute__((shared)) double as[B][B

AssertionError: 